In [82]:
# 
import pandas as pd, nltk, pymysql, re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [83]:
with pymysql.connect(user='root', passwd='ningunaok12OK@', database='AmazonSales') as connection:
    review : pd.DataFrame = pd.read_sql("SELECT * FROM Review;", connection)

review.head()

C:\Users\roylo\AppData\Local\Temp\ipykernel_3760\351007757.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  review : pd.DataFrame = pd.read_sql("SELECT * FROM Review;", connection)


,ReviewID,ReviewTitle,ReviewContent
0,R101TILZBOMQ6F,Good quality storage bag,https://m.media-amazon.com/images/I/7168Pizhei...
1,R101VJD80D1Z15,Good product product thanks usha.,Good product with good quality.
2,R103G2OV6OFA3Q,product review MI charger!!,i'm using it for 2 weeks now. No issues with t...
3,R10LLYRO2Z4E2G,Good product,https://m.media-amazon.com/images/I/71aTqzdBRd...
4,R10UNYZS2VXZ3G,Good product,Good product and keeps my charging organized. ...


# **`Sentiment Analysis`**

In [84]:
def isURL(text:str)->bool:
    """
    This function receives a string as an input and check whether it's an URL or not. Returns a boolean as the output.
    """
    return bool(re.compile(r'^(https?://)?([a-zA-Z0-9.-]+)(:[0-9]+)?(/.*)?$').match(text))

def Tokenize(text:str)->list[str]:
    """
    This function receives an string as an input. Then with help of the NLTK library we split the input into the phrases that the input has, after that,
    the punctuation symbols and stopwords are filtrated. Returns a list of strings.
    """
    ans : list = []
    for i in nltk.tokenize.sent_tokenize(text.lower()):
        j = nltk.tokenize.word_tokenize(re.sub('[^a-zA-Z]', " ", str(i)))
        stopwords : list[str] = [i for i in nltk.corpus.stopwords.words('english') if i not in ['good','super','ok']]
        j = [k for k in j if k not in stopwords]
        ans.append(' '.join(j))
    return ans

In [85]:
review['TitleTokenized'] = review['ReviewTitle'].apply(lambda n : Tokenize(n) if not isURL(n) else [])

review['ContentTokenized'] = review['ReviewContent'].apply(lambda n : Tokenize(n) if not isURL(n) else [])

review.head()

,ReviewID,ReviewTitle,ReviewContent,TitleTokenized,ContentTokenized
0,R101TILZBOMQ6F,Good quality storage bag,https://m.media-amazon.com/images/I/7168Pizhei...,[good quality storage bag],[]
1,R101VJD80D1Z15,Good product product thanks usha.,Good product with good quality.,[good product product thanks usha],[good product good quality]
2,R103G2OV6OFA3Q,product review MI charger!!,i'm using it for 2 weeks now. No issues with t...,"[product review mi charger, ]","[using weeks, issues product, satisfied produc..."
3,R10LLYRO2Z4E2G,Good product,https://m.media-amazon.com/images/I/71aTqzdBRd...,[good product],[]
4,R10UNYZS2VXZ3G,Good product,Good product and keeps my charging organized. ...,[good product],"[good product keeps charging organized, happy ..."


In [86]:
review['ContentTokenized'].iloc[4]

['good product keeps charging organized', 'happy buy']

In [87]:
sia : SentimentIntensityAnalyzer = SentimentIntensityAnalyzer()

def GetSentiment(l:list[str])->int:
    """
    This functions receives a list of string representing the review splitted into phrases. Each of the strings is passed through the 
    SentimentIntensityAnalyzer class from the NLTK library. Finally, the result is returned as:
        {
        "pos" : 2,
        "neu" : 1,
        "neg" : 0
        }
    """
    if not l:
        return 1

    sentiments = []
    for i in l:
        sentiment = sia.polarity_scores(i)
        sentiment.pop('compound')
        sentiments.append(max(sentiment, key=sentiment.get))
    
    sentimentsCount = dict(sorted(dict(Counter(sentiments)).items(), key=lambda n : n[1], reverse=True))

    maxsentimentsCount = max(sentimentsCount.values())
    if list(sentimentsCount.values()).count(maxsentimentsCount) == 1:
        ans : str = next(iter(sentimentsCount))
        return 0 if ans == 'neg' else 1 if ans == 'neu' else 2
    else:
        sentimentsCount : dict[str : int] = {i : j for i, j in sentimentsCount.items() if j == maxsentimentsCount}
        sentimentKeys : list = list(sentimentsCount.keys())
        if len(sentimentKeys) == 3:
            return 1
        else:
            if 'pos' in sentimentKeys:
                if 'neu' in sentimentKeys:
                    return 2
                if 'neg' in sentimentKeys:
                    return 1
            if 'neg' in sentimentKeys:
                if 'neu' in sentimentKeys:
                    return 0
                if 'pos' in sentimentKeys:
                    return 1
            if 'neu' in sentimentKeys:
                if 'pos' in sentimentKeys:
                    return 2
                if 'neg' in sentimentKeys:
                    return 0

In [88]:
review['TitleSentiment'] = review['TitleTokenized'].apply(GetSentiment)

review['ContentSentiment'] = review['ContentTokenized'].apply(GetSentiment)

review

,ReviewID,ReviewTitle,ReviewContent,TitleTokenized,ContentTokenized,TitleSentiment,ContentSentiment
0,R101TILZBOMQ6F,Good quality storage bag,https://m.media-amazon.com/images/I/7168Pizhei...,[good quality storage bag],[],1,1
1,R101VJD80D1Z15,Good product product thanks usha.,Good product with good quality.,[good product product thanks usha],[good product good quality],2,2
2,R103G2OV6OFA3Q,product review MI charger!!,i'm using it for 2 weeks now. No issues with t...,"[product review mi charger, ]","[using weeks, issues product, satisfied produc...",0,1
3,R10LLYRO2Z4E2G,Good product,https://m.media-amazon.com/images/I/71aTqzdBRd...,[good product],[],2,1
4,R10UNYZS2VXZ3G,Good product,Good product and keeps my charging organized. ...,[good product],"[good product keeps charging organized, happy ...",2,2
...,...,...,...,...,...,...,...
992,RZFMNMJ8EIG87,101% fake lava usb,Its not orignal usb its 100%fake lava usb (in ...,[fake lava usb],[orignal usb fake lava usb product pictures la...,0,1
993,RZN676INI7CXB,Good,Works the purpose of OTG for samsung galaxy A7...,[],[works purpose otg samsung galaxy tab],1,1
994,RZPZS0APQWNRT,Good,Company improve heavy wait same thing 2 kg,[],[company improve heavy wait thing kg],1,1
995,RZRHOS2N9ZVJM,Nice,Ok,[],[],1,1
